In [ ]:
# default_exp core

# Core utilities

> Utility functions and input folder

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export

import os

base_folder = os.environ.get(
    "S4REFSIMTOOL_INPUT", "/global/cfs/cdirs/cmbs4/dm/dstool/input/"
)

In [ ]:
# export

simulations_observing_efficiency = dict(
    pole=dict(SAT=0.3931149305555555, LAT=0.321544537037037),
    chile=dict(SAT=0.8656164351851852, LAT=0.917306712962963),
)

In [ ]:
#export


def get_telescope(channel):
    """Identify if a channel is in SAT or LAT

    Parameters
    ----------
    channel : str
        channel tag, e.g. MFH1

    Returns
    -------
    telescope : str
        'SAT' or 'LAT'
    """
    return channel[-2] + "AT"